<a href="https://colab.research.google.com/github/MMBazel/LO_GenAI_Workshops/blob/main/HelloTaylorSwift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

HellowWorld based on this tutorial:

*   https://www.kaggle.com/code/tommyadams/fine-tuning-tinyllama
*   Model: https://huggingface.co/huggingartists/taylor-swift
*   Dataset: https://huggingface.co/datasets/huggingartists/taylor-swift
*   https://www.youtube.com/watch?v=OVqe6GTrDFM






# Install necessary packages

In [1]:
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install -i https://pypi.org/simple/ bitsandbytes
!pip install einops wandb -Uqqq

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 28.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.7 MB/s eta 0:00:00


In [2]:
!pip freeze > requirements.txt

In [3]:
import torch
import glob
import pandas as pd
import numpy as np
import re
from peft import get_peft_model, PeftConfig, PeftModel, LoraConfig, prepare_model_for_kbit_training
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, GenerationConfig
from trl import SFTTrainer
from datasets import Dataset, DatasetDict

/usr/local/lib/python3.10/dist-packages/bitsandbytes/cextension.py:31: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


### Login With Credentials

In [4]:
from huggingface_hub import notebook_login

notebook_login()

# Load Data

### Load small data subset

In [5]:
from datasets import load_dataset

dataset = load_dataset("mmbazel/Taylor-Swift-Example")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['album_name', 'track_title', 'track_n', 'lyric', 'line'],
        num_rows: 8358
    })
})

In [7]:
# Extracting the lyrics and meta-data from the dataset
train_data = dataset["train"]
lyrics = train_data["lyric"]
album_name = train_data["album_name"]
track_title = train_data["track_title"]
track_n = train_data["track_n"]
line = train_data["line"]

In [8]:
# Cleaning the lyrics
replace_with_space = ['\u2005', '\u200b', '\u205f', '\xa0', '-']
replace_letters = {'í':'i', 'é':'e', 'ï':'i', 'ó':'o', ';':',', ''':'\'', ''':'\'', ':':',', 'е':'e'}
remove_list = ['\)', '\(', '–','"','"', '"', '\[.*\]', '.*\|.*', '—']

In [9]:
cleaned_lyrics = []
for lyric in lyrics:
    cleaned_lyric = lyric
    for old, new in replace_letters.items():
        cleaned_lyric = cleaned_lyric.replace(old, new)
    for string in remove_list:
        cleaned_lyric = re.sub(string,'',cleaned_lyric)
    for string in replace_with_space:
        cleaned_lyric = re.sub(string,' ',cleaned_lyric)
    cleaned_lyrics.append(cleaned_lyric)

In [10]:
# Creating a new dataset with cleaned lyrics and meta-data columns
train_dataset = Dataset.from_dict({
    'album_name': album_name,
    'track_title': track_title,
    'track_n': track_n,
    'lyric': cleaned_lyrics,
    'line': line
})

### Split train-test set

In [11]:
# Splitting the dataset into training, validation, and testing sets
train_percentage = 0.9
validation_percentage = 0.07
test_percentage = 0.03

In [12]:
train_lyrics, validation_lyrics, test_lyrics = np.split(cleaned_lyrics, [int(len(cleaned_lyrics)*train_percentage), int(len(cleaned_lyrics)*(train_percentage + validation_percentage))])
train_album_name, validation_album_name, test_album_name = np.split(album_name, [int(len(album_name)*train_percentage), int(len(album_name)*(train_percentage + validation_percentage))])
train_track_title, validation_track_title, test_track_title = np.split(track_title, [int(len(track_title)*train_percentage), int(len(track_title)*(train_percentage + validation_percentage))])
train_track_n, validation_track_n, test_track_n = np.split(track_n, [int(len(track_n)*train_percentage), int(len(track_n)*(train_percentage + validation_percentage))])
train_line, validation_line, test_line = np.split(line, [int(len(line)*train_percentage), int(len(line)*(train_percentage + validation_percentage))])

In [13]:
datasets = DatasetDict(
    {
        'train': Dataset.from_dict({
            'lyric': list(train_lyrics),
            'album_name': list(train_album_name),
            'track_title': list(train_track_title),
            'track_n': list(train_track_n),
            'line': list(train_line)
        }),
        'validation': Dataset.from_dict({
            'lyric': list(validation_lyrics),
            'album_name': list(validation_album_name),
            'track_title': list(validation_track_title),
            'track_n': list(validation_track_n),
            'line': list(validation_line)
        }),
        'test': Dataset.from_dict({
            'lyric': list(test_lyrics),
            'album_name': list(test_album_name),
            'track_title': list(test_track_title),
            'track_n': list(test_track_n),
            'line': list(test_line)
        })
    }
)

In [14]:
datasets

DatasetDict({
    train: Dataset({
        features: ['lyric', 'album_name', 'track_title', 'track_n', 'line'],
        num_rows: 7522
    })
    validation: Dataset({
        features: ['lyric', 'album_name', 'track_title', 'track_n', 'line'],
        num_rows: 585
    })
    test: Dataset({
        features: ['lyric', 'album_name', 'track_title', 'track_n', 'line'],
        num_rows: 251
    })
})

### Review Example

# Load Models

In [15]:
# Loading the model with double quantization
model_name = "PY007/TinyLlama-1.1B-step-50K-105b"

model = AutoModelForCausalLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/4.40G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

In [16]:
# Apply dynamic quantization
quantized_model = torch.quantization.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [17]:
# Move the quantized model to the appropriate device
device = "cuda" if torch.cuda.is_available() else "cpu"
quantized_model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): DynamicQuantizedLinear(in_features=2048, out_features=2048, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (k_proj): DynamicQuantizedLinear(in_features=2048, out_features=256, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (v_proj): DynamicQuantizedLinear(in_features=2048, out_features=256, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (o_proj): DynamicQuantizedLinear(in_features=2048, out_features=2048, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): DynamicQuantizedLinear(in_features=2048, out_features=5632, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
          (up_proj): DynamicQuantizedLinear(in_features=2048, out_fe

In [18]:
# Creating tokenizer and defining the pad token
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [21]:
def generate_lyrics(query, model):
    encoding = tokenizer(query, return_tensors="pt").to(device)
    generation_config = GenerationConfig(max_new_tokens=250, pad_token_id = tokenizer.eos_token_id,repetition_penalty=1.3, eos_token_id = tokenizer.eos_token_id)
    outputs = model.generate(input_ids=encoding.input_ids, generation_config=generation_config)
    text_output = tokenizer.decode(outputs[0],skip_special_tokens=True)
    print('INPUT\n', query, '\n\nOUTPUT\n', text_output[len(query):])

In [24]:
test_lyrics[200:700]

array(['Look at each other like that',
       'When the words of a sister come back in whispers',
       'That prove she was not in fact what she seemed',
       'Not a twin from your dreams', "She's a crook who was caught",
       'That old familiar body ache',
       'The snaps from the same little breaks in your soul',
       "You know when it's time to go", 'Twenty years at your job',
       'Then the son of the boss gets the spot that was yours',
       'Or trying to stay for the kids',
       'When keeping it how it is will only break their hearts worse',
       'That old familiar body ache',
       'The snaps from the same little breaks in your soul',
       "You know when it's time to go",
       'Sometimes giving up is the strong thing',
       'Sometimes to run is the brave thing',
       'Sometimes walking out is the one thing',
       'That will find you the right thing',
       'Sometimes giving up is the strong thing',
       'Sometimes to run is the brave thing',
       

In [23]:
type(test_lyrics[200:700])

numpy.ndarray

In [22]:
generate_lyrics(test_lyrics[200:700], quantized_model)

ValueError: text input must be of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).

In [26]:
# Setting arguments for low-rank adaptation

model = prepare_model_for_kbit_training(model)

lora_alpha = 32 # The weight matrix is scaled by lora_alpha/lora_rank, so I set lora_alpha = lora_rank to remove scaling
lora_dropout = 0.05
lora_rank = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_rank,
    bias="none",  # setting to 'none' for only training weight params instead of biases
    task_type="CAUSAL_LM")

peft_model = get_peft_model(model, peft_config)

In [27]:
# Setting training arguments

output_dir = "mmbazel/tinyllama_experiment" # Model repo on your hugging face account where you want to save your model
per_device_train_batch_size = 3
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
save_strategy="steps"
save_steps = 10
logging_steps = 10
learning_rate = 2e-3
max_grad_norm = 0.3 # Sets limit for gradient clipping
max_steps = 200     # Number of training steps
warmup_ratio = 0.03 # Portion of steps used for learning_rate to warmup from 0
lr_scheduler_type = "cosine" # I chose cosine to avoid learning plateaus

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
    report_to='none'
)


In [28]:
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=train_data_seg,
    peft_config=peft_config,
    max_seq_length=500,
    dataset_text_field='text',
    tokenizer=tokenizer,
    args=training_arguments
)
peft_model.config.use_cache = False

NameError: name 'train_data_seg' is not defined